27/09/2022

# Add colored segmentation on video

Use this script to add colored annotations and predictions on given dataset movies.

Used to generate examples for midterm exams.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import imageio

import numpy as np
from PIL import Image

from data_processing_tools import get_processed_result
from in_out_tools import (load_annotations_ids,
                          load_movies_ids,
                          write_colored_events_videos_on_disk)

In [4]:
import logging
import sys

############################# configure logger #############################

# set verbosity
verbosity = 3 

level_map = {
    3: logging.DEBUG,
    2: logging.INFO,
    1: logging.WARNING,
    0: logging.ERROR,
}
log_level = level_map[verbosity]
log_handlers = (logging.StreamHandler(sys.stdout),)

logging.basicConfig(
    level=log_level,
    format="[{asctime}] [{levelname:^8s}] [{name:^12s}] <{lineno:^4d}> -- {message:s}",
    style="{",
    datefmt="%H:%M:%S",
    handlers=log_handlers,
)

## Load movies, annotations & training predictions

Predictions need to be first saved running `load_trainings_predict.py`.

In [5]:
# assuming these are movie IDs whose predictions are available...
movie_ids_train = ["01","02","03","04","06","07","08","09",
                            "11","12","13","14","16","17","18","19",
                            "21","22","23","24","27","28","29",
                            "30","33","35","36","38","39",
                            "41","42","43","44","46"]

movie_ids_test  = ["05","10","15","20","25","32","34","40","45"]

#movie_ids = movie_ids_train+movie_ids_test
movie_ids = movie_ids_test

In [6]:
training_name = 'final_model_log_softmax'
epoch = 100000

In [7]:
data_dir = os.path.join("..","data","sparks_dataset")

preds_dir = os.path.join("trainings_validation",
                         training_name)

out_dir = os.path.join("trainings_validation",
                       training_name,
                       "colored_segmentation")
os.makedirs(out_dir, exist_ok=True)

In [8]:
### Load movies
xs = load_movies_ids(data_folder=data_dir,
                     ids=movie_ids,
                     names_available = True,
                     movie_names = 'video')


[14:17:34] [WARNING ] [tifffile.tifffile] <16549> -- TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'
[14:17:34] [WARNING ] [tifffile.tifffile] <16549> -- TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


In [9]:

### Load annotations -> using current annotations in dataset directory
ys = load_annotations_ids(data_folder=data_dir,
                          ids=movie_ids,
                          mask_names="class_label")

### Load annotated event instances
annotated_events = load_annotations_ids(data_folder=data_dir,
                                        ids=movie_ids,
                                        mask_names="event_label"
                                        )

### Load predictions
# Predictions created from selected model
sparks_filenames = {movie_id: os.path.join(preds_dir, training_name+"_"+str(epoch)+"_"+movie_id+"_sparks.tif") for movie_id in movie_ids}
puffs_filenames = {movie_id: os.path.join(preds_dir, training_name+"_"+str(epoch)+"_"+movie_id+"_puffs.tif") for movie_id in movie_ids}
waves_filenames = {movie_id: os.path.join(preds_dir, training_name+"_"+str(epoch)+"_"+movie_id+"_waves.tif") for movie_id in movie_ids}

sparks = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in sparks_filenames.items()}
puffs = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in puffs_filenames.items()}
waves = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in waves_filenames.items()}

## Get predictions segmentation

In [10]:
from data_processing_tools import sparks_connectivity_mask

In [11]:
# parameters for raw prediction processing

sigma = 3

# spark instances detection parameters
pixel_size = 0.2  # 1 pixel = 0.2 um x 0.2 um
min_dist_xy = round(1.8 / pixel_size) # min distance in space
time_frame = 6.8  # 1 frame = 6.8 ms
min_dist_t = round(20 / time_frame)  # min distance in time

min_dist_xy = min_dist_xy
min_dist_t = min_dist_t
conn_mask = sparks_connectivity_mask(min_dist_xy=min_dist_xy,
                                     min_dist_t=min_dist_t)

# TODO: use better parameters !!!
pixel_size = 0.2
spark_min_width = 3
spark_min_t = 3
puff_min_t = 5
wave_min_width = round(15 / pixel_size)

# connectivity for event instances detection
connectivity = 26

# maximal gap between two predicted puffs or waves that belong together
max_gap = 2  # i.e., 2 empty frames

In [12]:
preds_instances = {}
preds_segmentation = {}

In [13]:
from data_processing_tools import dict_to_int_mask

In [14]:
for movie_id in movie_ids:
    preds_instances[movie_id], preds_segmentation[movie_id], _ = get_processed_result(
            sparks=sparks[movie_id],
            puffs=puffs[movie_id],
            waves=waves[movie_id],
            xs=xs[movie_id],
            conn_mask=conn_mask,
            connectivity=connectivity,
            max_gap=max_gap,
            sigma=sigma,
            wave_min_width=wave_min_width,
            puff_min_t=puff_min_t,
            spark_min_t=spark_min_t,
            spark_min_width=spark_min_width,
            training_mode=False,
            debug=True
    )
    
    # convert dict of prends to int masks
    preds_segmentation[movie_id] = dict_to_int_mask(preds_segmentation[movie_id])
    preds_instances[movie_id] = sum(preds_instances[movie_id].values())

[14:17:50] [ DEBUG  ] [data_processing_tools] <448 > -- Events detection threshold: 0.498
[14:17:55] [ DEBUG  ] [data_processing_tools] <514 > -- Number of sparks detected by nonmaxima suppression: 139
[14:18:09] [ DEBUG  ] [data_processing_tools] <875 > -- Time for removing small events: 11.51 s
[14:18:09] [ DEBUG  ] [data_processing_tools] <448 > -- Events detection threshold: 0.498
[14:18:14] [ DEBUG  ] [data_processing_tools] <514 > -- Number of sparks detected by nonmaxima suppression: 26
[14:18:19] [ DEBUG  ] [data_processing_tools] <875 > -- Time for removing small events: 3.80 s
[14:18:19] [ DEBUG  ] [data_processing_tools] <448 > -- Events detection threshold: 0.498
[14:18:25] [ DEBUG  ] [data_processing_tools] <514 > -- Number of sparks detected by nonmaxima suppression: 30
[14:18:29] [ DEBUG  ] [data_processing_tools] <875 > -- Time for removing small events: 3.56 s
[14:18:30] [ DEBUG  ] [data_processing_tools] <448 > -- Events detection threshold: 0.498
[14:18:35] [ DEBUG  

## Add segmentation to original movies

### save original movie in RGB without ignored frames for comparison

In [15]:
# parameters
ignore_frames = 6

In [15]:
for movie_id in movie_ids:
    print("Processing video", movie_id, "...")
    
    # convert movie to rgb and save on disk
    movie_rgb = np.copy(xs[movie_id])
    movie_rgb = 255*(movie_rgb/movie_rgb.max())
    movie_rgb = [Image.fromarray(frame).convert('RGB') for frame in movie_rgb]
    if ignore_frames > 0:
        movie_rgb = [np.array(frame) 
                     for frame in movie_rgb[ignore_frames:-ignore_frames]]
    else:
        movie_rgb = [np.array(frame) for frame in movie_rgb]
    
    imageio.volwrite(os.path.join(out_dir, movie_id+"_original_movie.tif"),
                     movie_rgb)

Processing video 01 ...
Processing video 02 ...
Processing video 03 ...
Processing video 04 ...
Processing video 06 ...
Processing video 07 ...
Processing video 08 ...
Processing video 09 ...
Processing video 11 ...
Processing video 12 ...
Processing video 13 ...
Processing video 14 ...
Processing video 16 ...
Processing video 17 ...
Processing video 18 ...
Processing video 19 ...
Processing video 21 ...
Processing video 22 ...
Processing video 23 ...
Processing video 24 ...
Processing video 27 ...
Processing video 28 ...
Processing video 29 ...
Processing video 30 ...
Processing video 33 ...
Processing video 35 ...
Processing video 36 ...
Processing video 38 ...
Processing video 39 ...
Processing video 41 ...
Processing video 42 ...
Processing video 43 ...
Processing video 44 ...
Processing video 46 ...
Processing video 05 ...
Processing video 10 ...
Processing video 15 ...
Processing video 20 ...
Processing video 25 ...
Processing video 32 ...
Processing video 34 ...
Processing video

### add transparent segmentation of classes on original movies

In [16]:
# parameters
ignore_frames = 6
transparency = 70

In [17]:
for movie_id in movie_ids:
    print("Processing video", movie_id, "...")
    
    base_fn = training_name+"_"+str(epoch)+"_"+movie_id

    write_colored_events_videos_on_disk(movie=xs[movie_id],
                                        events_mask=preds_segmentation[movie_id],
                                        out_dir=out_dir,
                                        movie_fn=base_fn+"_colored_classes_preds",
                                        transparency=transparency,
                                        ignore_frames=ignore_frames,
                                        white_bg=False,
                                        instances=False)
    
    write_colored_events_videos_on_disk(movie=xs[movie_id],
                                        events_mask=ys[movie_id],
                                        out_dir=out_dir,
                                        movie_fn=base_fn+"_colored_classes_labels",
                                        transparency=transparency,
                                        ignore_frames=ignore_frames,
                                        white_bg=False,
                                        instances=False)

Processing video 05 ...
Processing video 10 ...
Processing video 15 ...
Processing video 20 ...
Processing video 25 ...
Processing video 32 ...
Processing video 34 ...
Processing video 40 ...
Processing video 45 ...


### add transparent segmentation of instances of events on movies

In [18]:
# parameters
ignore_frames = 6
transparency = 90

In [19]:
for movie_id in movie_ids:
    print("Processing video", movie_id, "...")
    
    base_fn = training_name+"_"+str(epoch)+"_"+movie_id

    # write predictions on disk
    write_colored_events_videos_on_disk(movie=xs[movie_id],
                                    events_mask=preds_instances[movie_id],
                                    out_dir=out_dir,
                                    movie_fn=base_fn+"_colored_instances_preds",
                                    transparency=transparency,
                                    ignore_frames=ignore_frames,
                                    white_bg=False,
                                    instances=True)
    # write labels on disk
    write_colored_events_videos_on_disk(movie=xs[movie_id],
                                    events_mask=annotated_events[movie_id],
                                    out_dir=out_dir,
                                    movie_fn=base_fn+"_colored_instances_labels",
                                    transparency=transparency,
                                    ignore_frames=ignore_frames,
                                    white_bg=False,
                                    instances=True)

Processing video 05 ...
Processing video 10 ...
Processing video 15 ...
Processing video 20 ...
Processing video 25 ...
Processing video 32 ...
Processing video 34 ...
Processing video 40 ...
Processing video 45 ...


### create masks where annotations are denoted by border and preds are transparent on original movie

In [20]:
# parameters
ignore_frames = 6
transparency = 70

In [21]:
for movie_id in movie_ids:
    print("Processing video", movie_id, "...")
    
    base_fn = training_name+"_"+str(epoch)+"_"+movie_id

    write_colored_events_videos_on_disk(movie=xs[movie_id],
                                        events_mask=preds_segmentation[movie_id],
                                        out_dir=out_dir,
                                        movie_fn=base_fn+"_colored_preds_and_labels",
                                        transparency=transparency,
                                        ignore_frames=ignore_frames,
                                        white_bg=False,
                                        instances=False,
                                        label_contours=True,
                                        label_mask=ys[movie_id])

Processing video 05 ...
Processing video 10 ...
Processing video 15 ...
Processing video 20 ...
Processing video 25 ...
Processing video 32 ...
Processing video 34 ...
Processing video 40 ...
Processing video 45 ...


### create masks with colored segmentation and white background (not updated)

In [11]:
for movie_id in movie_ids:
    print("Processing video", movie_id, "...")
    
    base_fn = training_name+"_"+str(epoch)+"_"+movie_id+"_white_bg"
    
    # write predictions on disk
    write_colored_events_videos_on_disk(movie=xs[movie_id],
                                       classes_mask=preds[movie_id],
                                       out_dir=out_dir,
                                       movie_fn=base_fn+"_colored_preds",
                                       transparency=transparency,
                                       ignore_frames=ignore_frames,
                                       white_bg=True)
    
    # write labels on disk
    write_colored_events_videos_on_disk(movie=xs[movie_id],
                                       classes_mask=ys[movie_id],
                                       out_dir=out_dir,
                                       movie_fn=base_fn+"_colored_labels",
                                       transparency=transparency,
                                       ignore_frames=ignore_frames,
                                       white_bg=True)

Processing video 01 ...
Processing video 02 ...
Processing video 03 ...
Processing video 04 ...
Processing video 06 ...
Processing video 07 ...
Processing video 08 ...
Processing video 09 ...
Processing video 11 ...
Processing video 12 ...
Processing video 13 ...
Processing video 14 ...
Processing video 16 ...
Processing video 17 ...
Processing video 18 ...
Processing video 19 ...
Processing video 21 ...
Processing video 22 ...
Processing video 23 ...
Processing video 24 ...
Processing video 27 ...
Processing video 28 ...
Processing video 29 ...
Processing video 30 ...
Processing video 33 ...
Processing video 35 ...
Processing video 36 ...
Processing video 38 ...
Processing video 39 ...
Processing video 41 ...
Processing video 42 ...
Processing video 43 ...
Processing video 44 ...
Processing video 46 ...
Processing video 05 ...
Processing video 10 ...
Processing video 15 ...
Processing video 20 ...
Processing video 25 ...
Processing video 32 ...
Processing video 34 ...
Processing video